In [ ]:
import ModelClassForPrediction as Model
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import r2_score

In [ ]:
#NN1500RS=Model.Model(PathToModel="/home/villarreal/Documents/STO_MD/ModelsAndData/OCT18/nvt.1500.RS.PBC.trainingsesh.18OCT.2021/",\
#    PathToData="/home/villarreal/Documents/STO_MD/ModelsAndData/OCT18/nvt.1500.RS.OCT10.2021_WITHPBC/RStrajectoryTest.npz",\
#        TestSetSize=3)

NN1500RS=Model.Model(PathToModel="/Users/davidvillarreal/Documents/PhD/DataSetsAndModels/nequip_training_RS_1500K_pbc/",\
    PathToData="/Users/davidvillarreal/Documents/PhD/DataSetsAndModels/1500K/NRS/trajectoryTest.npz",\
        TestSetSize=5)

In [ ]:
PredictedForces,PredictedEnergies,\
ActualForces,ActualEnergies \
=NN1500RS.PredictForcesAndEnergies()

n=PredictedForces.shape[-1]












# MAEOverAvg=np.mean(np.abs(ActualForces.reshape(-1,n))-np.abs(PredictedForces.reshape(-1,n)),axis=0)\
# /np.mean(np.abs(ActualForces.reshape(-1,n)),axis=0)

# MAEs=np.mean(np.abs(ActualForces.reshape(-1,n))-np.abs(PredictedForces.reshape(-1,n)),axis=0)

# MaxEOverAvg=np.max(np.abs(ActualForces.reshape(-1,n))-np.abs(PredictedForces.reshape(-1,n)),axis=0)\
# /np.mean(np.abs(ActualForces.reshape(-1,n)),axis=0)

# NN1500RSPerformance=pd.DataFrame({'ActualForces':ActualForces.reshape(-1,n).flatten(),'Energies':ActualEnergies,'MAEs':MAEs,'MAEOverAvg':MAEOverAvg,
#                                 'MaxEOverAvg':MaxEOverAvg})


##Absolute Forces versus Absolute Errors jointplot

In [ ]:
AEs=np.abs(ActualForces.reshape(-1,n).flatten()-PredictedForces.reshape(-1,n).flatten())

df=pd.DataFrame({"ActualForces":ActualForces.reshape(-1,n).flatten(),"AEs":AEs})
sns.set(font_scale=1.2)
g=sns.jointplot(data=df,x="AEs",y="ActualForces",height=6)

mae=df["AEs"].mean()
maeoveravgf=mae/np.abs(ActualForces.reshape(-1,n)).flatten().mean()
maxe=df["AEs"].max()
maxeoveravgf=df["AEs"].max()/np.abs(ActualForces.reshape(-1,n)).flatten().mean()


g.ax_joint.axvline(x=df["AEs"].mean(),linestyle='--',c='blue',label=f"MAE = {mae:.4f} \n MAE/AvgF = \
    {maeoveravgf*100:.4f} %")
g.ax_joint.axvline(x=maxe,linestyle='--',c='red',label=f"MaxE = {mae:.4f} \n MaxE/AvgF = \
    {maxeoveravgf*100:.4f} %")
g.set_axis_labels('AEs (eV/Ang)', 'ActualForces (eV/Ang)', fontsize=16)
g.fig.legend()


R2=r2_score(ActualForces.reshape(-1,n).flatten(),PredictedForces.reshape(-1,n).flatten())
plt.figure(figsize=(5,5))
plt.plot(ActualForces.reshape(-1,n).flatten(),PredictedForces.reshape(-1,n).flatten(),'o',\
    label=f"R2 = {R2:.4f}, \n MAE = {mae:.4f} \n MAEOverAvgF = {maeoveravgf*100:.4f} % \n MaxEOverAvgF = {maxeoveravgf*100:.4f} %")
plt.xlabel("ActualForces (eV/Ang)"); plt.ylabel("PredictedForces (eV/Ang)")
plt.legend()

plt.figure(figsize=(5,5))
df
mae=df["AEs"].mean()
maeoveravgf=mae/np.abs(ActualForces.reshape(-1,n)).flatten().mean()
maxe=df["AEs"].max()
maxeoveravgf=df["AEs"].max()/np.abs(ActualForces.reshape(-1,n)).flatten().mean()


##### Now for energies

mae=np.abs(ActualEnergies.flatten()-PredictedEnergies.flatten()).mean()
R2=r2_score(ActualEnergies,PredictedEnergies)
plt.figure(figsize=(5,5))
plt.plot(ActualEnergies,PredictedEnergies,'o',label=f"R2 = {R2:.4f}\n MAE = {mae:.4f}")
plt.xlabel("ActualEnergies (eV)"); plt.ylabel("PredictedEnergies (eV)")
plt.legend()



AEs=np.abs(ActualEnergies.reshape(-1,n).flatten()-PredictedEnergies.reshape(-1,n).flatten())

df=pd.DataFrame({"ActualEnergies":ActualEnergies.reshape(-1,n).flatten(),"AEs":AEs})
sns.set(font_scale=1.2)
g=sns.jointplot(data=df,x="AEs",y="ActualEnergies",height=6)


mae=df["AEs"].mean()
maeoveravgf=mae/np.abs(ActualEnergies).flatten().mean()
maxe=df["AEs"].max()
maxeoveravgf=df["AEs"].max()/np.abs(ActualForces).flatten().mean()


g.ax_joint.axvline(x=df["AEs"].mean(),linestyle='--',c='blue',label=f"MAE = {mae:.4f} \n MAE/AvgE = \
    {maeoveravgf*100:.4f} %")
g.ax_joint.axvline(x=maxe,linestyle='--',c='red',label=f"MaxE = {mae:.4f} \n MaxE/AvgE = \
    {maxeoveravgf*100:.4f} %")
g.set_axis_labels('AEs (eV)', 'ActualForces (eV)', fontsize=16)
g.fig.legend()

Training and Validation Curves

In [ ]:
results_path='/Users/davidvillarreal/Documents/PhD/DataSetsAndModels/nequip_training_RS_1500K_pbc/tutorial-results/requeue'
# Load logged results
results = np.loadtxt(os.path.join(results_path, 'metrics_epoch.csv'), skiprows=1, delimiter=',')

# Determine time axis
time = (results[:,1]-results[0,1])/(60*60)

# Load the validation MAEs
energy_mae = results[:,14]
forces_mae = results[:,13]
train_loss=results[:,5]
val_loss=results[:,11]
# Get final validation errors
print('Validation MAE:')
print('    energy: {:10.3f} eV/atom'.format(energy_mae[-1]))
print('    forces: {:10.3f} eV/atom/Ang'.format(forces_mae[-1]))

# Construct figure
plt.figure(figsize=(14,5))

# Plot energies
#plt.subplot(1,2,1)
plt.plot(time, energy_mae)
plt.title('Energy')
plt.ylabel('MAE [eV/atom]')
plt.xlabel('Time [s]')

plt.figure(figsize=(14,5))

# Plot forces
#plt.subplot(1,2,2)
plt.plot(time, forces_mae)
plt.title('Forces')
plt.ylabel('MAE [eV/atom/Ang]')
plt.xlabel('Time [s]')

plt.show()
#plt.savefig('MAEs.png')
#END OF MAEs
##################################################################################################
# Construct figure
plt.figure(figsize=(14,5))
# Plot train and val loss
#plt.subplot(1,2,1)
plt.semilogy(time, train_loss,label='train loss')
plt.semilogy(time, val_loss,label='val loss')
plt.title('Train and val loss')
plt.ylabel('A.u.')
plt.xlabel('Time [s]')
plt.legend()


